# Crawling & Pre-processing Twitter data

## Download

In [1]:
# -*- coding: utf-8 -*-
import tweepy as tw
import sys,jsonpickle

# find it yourself okay?
consumer_key = ''
consumer_secret = ''
 
qry='jogja'
maxTweets = 30
tweetsPerQry = 30
fName='tweepy ' +qry+ '.json'
 
auth = tw.AppAuthHandler(consumer_key,consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    sys.exit('Authentication failed')
 
sinceId=None;max_id=-1;tweetCount=0
print("Start downloading... {0} tweets".format(maxTweets))
with open(fName,'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,result_type="popular")
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,since_id=sinceId,result_type="popular")
            else:
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1),result_type="popular")
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1),since_id=sinceId,result_type="popular")
            if not new_tweets:
                print('No more Tweets found with Query="{0}"'.format(qry));break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
            tweetCount+=len(new_tweets)
            sys.stdout.write("\r");sys.stdout.write("Tweets has been saved: %.0f" %tweetCount);sys.stdout.flush()
            max_id=new_tweets[-1].id
        except tw.TweepError as e:
            print("some error : " + str(e));break # Aya error, keluar
print ('\nDone! {0} tweets saved at "{1}"'.format(tweetCount,fName))

Start downloading... 30 tweets
Tweets has been saved: 30
Done! 30 tweets saved at "tweepy jogja.json"


## Preprocess

In [40]:
# -*- coding: utf-8 -*-
import json,re
file = open('tweepy ' +qry+ '.json') 
lines = file.readlines()

hasil=open("tweepy_preprocessed.csv",'w')
for i in lines:
#     print(json.loads(i.strip())['text'].lower())
    tweet = json.loads(i.strip())['text'].lower()
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*','', tweet).replace('\n', ' ')
    hasil.write(str(tweet.encode('utf-8'))+'\n')
    
hasil.close()

## Labeling

In [5]:
import json,re
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict

In [6]:
import pandas as pd

In [ ]:
# tweets = pd.read_table("Book7.csv", header=None, names=['tweet','label'])

In [8]:
tweets = pd.read_csv("Book7.csv", sep=';', header=None, names=["text","label"])

In [9]:
tweets

,text,label
0,breaking news : pss sleman tahan imbang kalten...,1
1,wkt pulang dr jogja mau share foto ini tapi lu...,0
2,link live streaming useetv liga 2 2018 kalteng...,0
3,"siaran pers 24 november 2018, mengenai kejadia...",0
4,"breaking news : waspada, empat kali guguran la...",-1
5,pelatih pss sleman syukuri hasil imbang lawan ...,1
6,selamat hari guru nasional. tetaplah menjadi p...,1
7,keutamaan dan doa puasa sunnah senin kamis bag...,1
8,pss sleman tahan imbang kalteng putra di palan...,1
9,"raih satu poin di kandang kalteng putra, ini k...",1


In [10]:
tweets.label.value_counts()

 0    15
 1    11
-1     4
Name: label, dtype: int64

In [11]:
X = tweets.text
y = tweets.label
print(X.shape)
print(y.shape)

(30,)
(30,)


## Split Train Test

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(27,)
(3,)
(27,)
(3,)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
# X = vectorizer.fit_transform(tweets)
# vectorizer.get_feature_names()

In [17]:
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)

In [18]:
X_test_dtm = vect.transform(X_test)

## NBC

In [19]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [20]:
%time nb.fit(X_train_dtm, y_train)

Wall time: 997 µs


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
y_pred_class = nb.predict(X_test_dtm)

In [22]:
y_pred_class

array([1, 0, 0], dtype=int64)

In [23]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.3333333333333333

In [24]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[0, 1, 0],
       [0, 0, 0],
       [0, 1, 1]], dtype=int64)

# Clustering

## KMeans

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [26]:
documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

In [27]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [28]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [29]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 google
 cat
 best
 climbing
 ninja
 incredible
 app
 translate
 impressed
 map
Cluster 1:
 eating
 kitty
 little
 came
 restaurant
 play
 ve
 feedback
 face
 extension


In [30]:
Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print("Prediction", prediction)

Prediction [0]


In [31]:
Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print("Prediction", prediction)

Prediction [0]


In [32]:
Y = vectorizer.transform(["photo restaurant mcdonald"])
prediction = model.predict(Y)
print("Prediction", prediction)

Prediction [0]


In [33]:
Y = vectorizer.transform(["take your photo off the wall"])
prediction = model.predict(Y)
print("Prediction", prediction)

Prediction [0]
